# Importando e baixando bibliotecas

In [ ]:
!pip install emoji
!pip install emoji_data_python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.4.0-py2.py3-none-any.whl size=350809 sha256=756919ad3e11f27a761115c0736943d0d78d3a7380c6087f0b1ca81f9d3eee0e
  Stored in directory: /root/.cache/pip/wheels/79/29/1c/234cae4632803c2ba4a76a71a679eb1383cf590775714e2a21
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.8/105.8 kB 8.9 MB/s eta 0:00:00


In [ ]:
!pip install spacy
!python -m spacy download pt_core_news_lg 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-25 01:13:48.135513: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 01:13:49.252456: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-25 01:13:51.240916: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-25 01:13:51.241505: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA 

import numpy as np
import unicodedata
import re
import emoji
import spacy 
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec

In [2]:
import pandas as pd
import emoji
import numpy as np
import spacy
import re
import unicodedata

In [3]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import matplotlib.pyplot as plt

In [4]:
gensim.__version__

'4.3.1'

# Pipeline para pré processamento

In [5]:
#Erros gramaticais e abreviações
#Dicionario de abreviações para substituir 
def substituir_abreviacoes(frase):
    dicionario_girias = {
    'vc': 'voce',
    'vcs':'voce',
    'Vc': 'voce',
    'pq': 'porque',
    'tbm': 'tambem',
    'q': 'que',
    'td': 'tudo',
    'blz': 'beleza',
    'flw': 'falou',
    'kd': 'cade',
    'to': 'estou',
    'mt': 'muito',
    'cmg': 'comigo',
    'ctz': 'certeza',
    'jah': 'ja',
    'naum': 'nao',
    'ta': 'esta',
    'eh': 'e',
    'vlw': 'valeu',
    'p': 'para',
    'qnd': 'quando',
    'msm': 'mesmo',
    'fzr': 'fazer',
    'agr': 'agora',
    'btgpactual': 'btg',
    'pactual': 'btg',
    }
    palavras = frase.split()
    for i, palavra in enumerate(palavras):
        if palavra in dicionario_girias:
            palavras[i] = dicionario_girias[palavra]
    return ' '.join(palavras)

In [6]:
#Retirada de caracteres especias
def remover_caracteres_especiais(texto):
    texto_sem_acentos = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto_sem_especiais = re.sub('[^a-zA-Z0-9 \\\]', '', texto_sem_acentos)
    return texto_sem_especiais

In [7]:
def pipeline(texto):
    # Substituição de emojis
    texto = emoji.demojize(texto, delimiters=(' ', ' '), language='pt')

    # Retirada de caracteres especiais
    texto = remover_caracteres_especiais(texto)
    
    # Muda abreviações e variantes de palavras como btg
    
    texto = substituir_abreviacoes(texto)


    # Tokenização e lower
    nlp = spacy.load("pt_core_news_lg")
    doc = nlp(texto.lower())

    # Retirada de stopwords e pontuações, lematização
    tokens_new = []
    stop_words = ['eu', 'tu', 'ele', 'ela', 'nós', 'nos', 'vós', 'vos', 'eles', 'elas', 
                  'isso', 'isto', 'aquilo', 'algum', 'alguma', 'alguns', 'algumas',
                  'outro', 'outra', 'outros', 'outras', 'tão', 'tal', 'tanto', 'tanta', 'tantos', 'tantas', 
                  'seu', 'sua', 'seus', 'suas', 'dele', 'dela', 'deles', 'delas', 
                  'quem', 'qual', 'quais', 'que', 'onde', 'como', 'e','um', 'as', 'no',
                  'para', 'por', 'com', 'sem', 'sob', 'sobre', 'de', 'da', 'desde', 'em', 'entre', 'porque',
                  'á', 'a', 'o', 'ola', 'olá', 'pra', 'para', 'bemvindo', 'benvindo', 'bem-vindo', 'bemvindos', 'aqui', 'vai', 'na', 'no']
    pontuacoes = ['.', ',', ';', ':', '!', '?', '-', '—', '_', '/', '\\', '(', ')', '[', ']', '{', '}', '"', "'", '`', '~', '^', '@', '#', '%', '&', '*']

    for token in doc:
        if token.text not in stop_words and token.text not in pontuacoes:
            tokens_new.append(token.lemma_)

    return tokens_new

In [ ]:
def pipeline_sentece(texto):
    # Substituição de emojis
    texto = emoji.demojize(texto, delimiters=(' ', ' '), language='pt')

    # Retirada de caracteres especiais
    texto = remover_caracteres_especiais(texto)
    
    # Muda abreviações e variantes de palavras como btg
    
    texto = substituir_abreviacoes(texto)


    # Tokenização e lower
    nlp = spacy.load("pt_core_news_lg")
    doc = nlp(texto.lower())

    # Retirada de stopwords e pontuações, lematização
    tokens_new = []
    stop_words = ['eu', 'tu', 'ele', 'ela', 'nós', 'nos', 'vós', 'vos', 'eles', 'elas', 
                  'isso', 'isto', 'aquilo', 'algum', 'alguma', 'alguns', 'algumas',
                  'outro', 'outra', 'outros', 'outras', 'tão', 'tal', 'tanto', 'tanta', 'tantos', 'tantas', 
                  'seu', 'sua', 'seus', 'suas', 'dele', 'dela', 'deles', 'delas', 
                  'quem', 'qual', 'quais', 'que', 'onde', 'como', 'e','um', 'as', 'no',
                  'para', 'por', 'com', 'sem', 'sob', 'sobre', 'de', 'da', 'desde', 'em', 'entre', 'porque',
                  'á', 'a', 'o', 'ola', 'olá', 'pra', 'para', 'bemvindo', 'benvindo', 'bem-vindo', 'bemvindos', 'aqui', 'vai', 'na', 'no']
    pontuacoes = ['.', ',', ';', ':', '!', '?', '-', '—', '_', '/', '\\', '(', ')', '[', ']', '{', '}', '"', "'", '`', '~', '^', '@', '#', '%', '&', '*']

    for token in doc:
        if token.text not in stop_words and token.text not in pontuacoes:
            tokens_new.append(token.lemma_)

    return ''.join(tokens_new)

In [ ]:
pipeline_sentence('Gosto de Correr, Ana de Armas')

In [ ]:
pipeline('Gosto de Correr, Ana de Armas')

# Data Frame

In [90]:
df = pd.read_csv("C:\\Users\\Rodrigo.INTELI\\Downloads\\2-base_10052023.xlsx - Sheet1.csv")
df = df.rename(columns={'"autor"': 'autor', '"texto"': 'texto', '"sentimento"': 'sentimento'})
df_texto = df[['autor', 'texto','sentimento' ]].dropna().drop(df[df['autor'] == 'btgpactual'].index)
#Colocando os textos em um corpus
arraytextos = df_texto['texto'].values.flatten().astype(str)
texto = ''.join(arraytextos)

In [11]:
df_texto

,autor,texto,sentimento
0,v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL
1,winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL
2,marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL
3,lmviapiana,Minuto touro de ouro,POSITIVE
4,vanilson_dos,@ricktolledo Sim,NEUTRAL
...,...,...,...
12350,onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL
12351,amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL
12352,jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE
12353,moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL


# Executando o pipeline

In [12]:
df_texto['tokens_lema'] = df_texto['texto'].apply(pipeline)

In [13]:
df_texto.head()

,autor,texto,sentimento,tokens_lema
0,v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,"[confira, o, resultado, de o, nosso, fundo, me..."
1,winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,"[Alvarez, Marsal, estara, conosco, sportainmet..."
2,marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,"[repost, btg, With, makerepost, entenda, o, im..."
3,lmviapiana,Minuto touro de ouro,POSITIVE,"[minuto, touro, ouro]"
4,vanilson_dos,@ricktolledo Sim,NEUTRAL,"[Ricktolledo, sim]"


In [ ]:
#Fazer o modelo word2vec com api e sem api.
#Fazer a soma dos vetores, criar coluna sum.
#Perguntar pro gepeto, o que faço depois de vetorizar, no sentido de criar uma machine learning
#Modelo naive bayes
#Modelo Xgboost

# Word2Vec

In [15]:
tokens = df_texto['tokens_lema'].tolist()
# Crie o modelo Word2Vec
modelo_w2v = Word2Vec(tokens, vector_size=150, window=5, min_count=1, workers=4)

# O parâmetro 'size' define a dimensionalidade dos vetores de palavra
# O parâmetro 'window' define o tamanho máximo da janela de contexto
# O parâmetro 'min_count' define a contagem mínima de ocorrências para uma palavra ser considerada
# O parâmetro 'workers' define o número de threads para treinamento paralelo (aumenta a velocidade)


In [102]:
modelo_w2v.save("C:\\Users\\Rodrigo.INTELI\\Downloads\\modelo\\modelo_word2vec.bin") # salvando modelo

In [104]:
modelo = Word2Vec.load("C:\\Users\\Rodrigo.INTELI\\Downloads\\modelo\\modelo_word2vec.bin") # load no modelo

In [17]:
# Função para vetorizar um token
def vetorizar_token(token):
    vetor = np.zeros(modelo_w2v.vector_size) #incializa vetor de zeros com a mesma dimensão
    if token in modelo_w2v.wv: #verifica se a palavra está no word2vec treinado
        vetor = modelo_w2v.wv[token] #adiciona o valor do vetor
    return vetor

# Função para vetorizar uma frase
def vetorizar_frase(frase):
    vetores_tokens = [vetorizar_token(token) for token in frase] # verifica cada token da lista
    return np.sum(vetores_tokens, axis=0) # retorna a soma dos vetores

# Aplicar a função 'vetorizar_frase' a todas as frases
df_texto['vetores'] = df_texto['tokens_lema'].apply(vetorizar_frase)
df_texto

,autor,texto,sentimento,tokens_lema,vetores
0,v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,"[confira, o, resultado, de o, nosso, fundo, me...","[24.175386, -32.35245, -19.241129, -8.272617, ..."
1,winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,"[Alvarez, Marsal, estara, conosco, sportainmet...","[1.3696473, -17.072432, -9.895778, -2.7429254,..."
2,marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,"[repost, btg, With, makerepost, entenda, o, im...","[12.244981, -20.071892, -7.7097063, -5.488888,..."
3,lmviapiana,Minuto touro de ouro,POSITIVE,"[minuto, touro, ouro]","[0.059000924, 0.20170294, -0.35151416, -0.0123..."
4,vanilson_dos,@ricktolledo Sim,NEUTRAL,"[Ricktolledo, sim]","[0.609844, -0.014483982, -0.49792445, -0.50369..."
...,...,...,...,...,...
12350,onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,"[um, noite, encontro, muito, conhecimento, nos...","[13.11174, -5.5462904, -5.497891, -11.828852, ..."
12351,amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,"[erro, financeiro, eliminar, antes, de o, 30, ...","[39.5939, -11.322476, -47.345585, -30.336466, ..."
12352,jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,"[estar, muito, grato, todo, esforco, voce, eu,...","[12.066106, 6.0471125, -8.153074, -10.89392, 3..."
12353,moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,[dorsodamaocomdedoindicadorapontandoparaadirei...,"[36.842133, 1.6632437, -21.75585, -26.044128, ..."


In [36]:
df_texto['soma_vetores'] = df_texto['vetores'].apply(lambda x: np.sum(x, axis=0))
df_texto

,autor,texto,sentimento,tokens_lema,vetores,soma_vetores
0,v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,"[confira, o, resultado, de o, nosso, fundo, me...","[24.175386, -32.35245, -19.241129, -8.272617, ...",-20.574974
1,winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,"[Alvarez, Marsal, estara, conosco, sportainmet...","[1.3696473, -17.072432, -9.895778, -2.7429254,...",-44.773376
2,marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,"[repost, btg, With, makerepost, entenda, o, im...","[12.244981, -20.071892, -7.7097063, -5.488888,...",3.235275
3,lmviapiana,Minuto touro de ouro,POSITIVE,"[minuto, touro, ouro]","[0.059000924, 0.20170294, -0.35151416, -0.0123...",-0.193069
4,vanilson_dos,@ricktolledo Sim,NEUTRAL,"[Ricktolledo, sim]","[0.609844, -0.014483982, -0.49792445, -0.50369...",-0.936380
...,...,...,...,...,...,...
12350,onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,"[um, noite, encontro, muito, conhecimento, nos...","[13.11174, -5.5462904, -5.497891, -11.828852, ...",96.413528
12351,amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,"[erro, financeiro, eliminar, antes, de o, 30, ...","[39.5939, -11.322476, -47.345585, -30.336466, ...",-234.310669
12352,jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,"[estar, muito, grato, todo, esforco, voce, eu,...","[12.066106, 6.0471125, -8.153074, -10.89392, 3...",33.128986
12353,moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,[dorsodamaocomdedoindicadorapontandoparaadirei...,"[36.842133, 1.6632437, -21.75585, -26.044128, ...",89.220032


# Criando um arquivo csv com a coluna lematização, vetores e soma de vetores

In [105]:
df_texto['tokens_lema'] = np.array(df_texto['tokens_lema'])
df_texto['sentimento'] = df['sentimento'].map({'NEUTRAL': 0, 'POSITIVE': 1, 'NEGATIVE': -1})
df_texto

,autor,texto,sentimento,tokens_lema,vetores,soma_vetores
0,v8_capital,Confira os resultados dos nossos fundos no mês...,0,"[confira, o, resultado, de o, nosso, fundo, me...","[24.175386, -32.35245, -19.241129, -8.272617, ...",-20.574974
1,winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,0,"[Alvarez, Marsal, estara, conosco, sportainmet...","[1.3696473, -17.072432, -9.895778, -2.7429254,...",-44.773376
2,marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,0,"[repost, btg, With, makerepost, entenda, o, im...","[12.244981, -20.071892, -7.7097063, -5.488888,...",3.235275
3,lmviapiana,Minuto touro de ouro,1,"[minuto, touro, ouro]","[0.059000924, 0.20170294, -0.35151416, -0.0123...",-0.193069
4,vanilson_dos,@ricktolledo Sim,0,"[Ricktolledo, sim]","[0.609844, -0.014483982, -0.49792445, -0.50369...",-0.936380
...,...,...,...,...,...,...
12350,onixcapital,Uma noite de encontros e muito conhecimento on...,0,"[um, noite, encontro, muito, conhecimento, nos...","[13.11174, -5.5462904, -5.497891, -11.828852, ...",96.413528
12351,amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,0,"[erro, financeiro, eliminar, antes, de o, 30, ...","[39.5939, -11.322476, -47.345585, -30.336466, ...",-234.310669
12352,jayipacderota,Estou muito grato por todo o esforço que você ...,1,"[estar, muito, grato, todo, esforco, voce, eu,...","[12.066106, 6.0471125, -8.153074, -10.89392, 3...",33.128986
12353,moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",0,[dorsodamaocomdedoindicadorapontandoparaadirei...,"[36.842133, 1.6632437, -21.75585, -26.044128, ...",89.220032


In [118]:
df_texto.to_excel('df_novo3.xlsx', index=False)

In [4]:
df_novo = pd.read_excel("C:\\Users\\Rodrigo.INTELI\\Downloads\\df_novo3.xlsx")
df_novo

,autor,texto,sentimento,tokens_lema,vetores,soma_vetores
0,v8_capital,Confira os resultados dos nossos fundos no mês...,0,"['confira', 'o', 'resultado', 'de o', 'nosso',...",[ 24.175386 -32.35245 -19.241129 -8.27...,-20.574974
1,winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,0,"['Alvarez', 'Marsal', 'estara', 'conosco', 'sp...",[ 1.36964726e+00 -1.70724316e+01 -9.89577770e+...,-44.773376
2,marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,0,"['repost', 'btg', 'With', 'makerepost', 'enten...",[ 12.244981 -20.071892 -7.7097063 -5.48...,3.235275
3,lmviapiana,Minuto touro de ouro,1,"['minuto', 'touro', 'ouro']",[ 0.05900092 0.20170294 -0.35151416 -0.012351...,-0.193069
4,vanilson_dos,@ricktolledo Sim,0,"['Ricktolledo', 'sim']",[ 0.609844 -0.01448398 -0.49792445 -0.503695...,-0.936380
...,...,...,...,...,...,...
12188,onixcapital,Uma noite de encontros e muito conhecimento on...,0,"['um', 'noite', 'encontro', 'muito', 'conhecim...",[ 13.11174 -5.5462904 -5.497891 -11.82...,96.413528
12189,amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,0,"['erro', 'financeiro', 'eliminar', 'antes', 'd...",[ 3.95938988e+01 -1.13224764e+01 -4.73455849e+...,-234.310669
12190,jayipacderota,Estou muito grato por todo o esforço que você ...,1,"['estar', 'muito', 'grato', 'todo', 'esforco',...",[ 12.066106 6.0471125 -8.153074 -10.89...,33.128986
12191,moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",0,['dorsodamaocomdedoindicadorapontandoparaadire...,[ 36.842133 1.6632437 -21.75585 -26.04...,89.220032


In [ ]:
df_vec = pd.read_csv('')

In [149]:
sentence_table = []
#Pecorre cada token e faz seu vetor correspondente
for sentence in df_texto['tokens_lema']:
    word_vectors = [modelo_w2v.wv[word] for word in sentence if word in modelo_w2v.wv]
    if len(word_vectors) > 0:
        sentence_vector = sum(word_vectors) / len(word_vectors)
    else:
        sentence_vector = [None] * 150 # Cria uma lista de 150 elementos None
    sentence_table.append((sentence, *sentence_vector[:150]))  # Adiciona apenas os primeiros 150 elementos do vetor

column_labels = ['tokens_lema']

#looping para adicionar a coluna vetor com seu número correspondente
for i in range(150):
    column_labels.append(f'vec{i+1}')
    
# Cria um novo DataFrame com os vetores de sentença
df_vec = pd.DataFrame(sentence_table, columns=column_labels)
# Define o índice do DataFrame df_vec como o mesmo índice de df_texto['sentimento']
df_vec.set_index(df_texto["sentimento"].index, inplace=True)
# Adiciona a coluna 'sentimento' ao DataFrame df_vec a partir de df_texto['sentimento']
df_vec['sentimento'] = df_texto["sentimento"]

# Salvando tabela com vetores e sentimento

In [20]:
df_vec.to_csv('df_vec.csv', index=False) #baixando tabela de vetores

In [21]:
df_vec = pd.read_csv("C:\\Users\\Rodrigo.INTELI\\Downloads\\df_vec.csv")

In [22]:
df_vec

,tokens_lema,vec1,vec2,vec3,vec4,vec5,vec6,vec7,vec8,vec9,...,vec142,vec143,vec144,vec145,vec146,vec147,vec148,vec149,vec150,sentimento
0,"['confira', 'o', 'resultado', 'de o', 'nosso',...",0.262776,-0.351657,-0.209143,-0.089920,0.160042,-0.323829,0.167901,0.557442,0.193186,...,0.095108,0.516315,0.196075,0.066405,0.206681,0.056883,-0.308131,0.319699,-0.315743,0
1,"['Alvarez', 'Marsal', 'estara', 'conosco', 'sp...",0.029775,-0.371140,-0.215126,-0.059629,0.000434,-0.177448,0.356608,0.857633,-0.027186,...,-0.080119,1.078803,0.284863,-0.010041,0.329369,-0.068786,-0.359836,0.238163,-0.509014,0
2,"['repost', 'btg', 'With', 'makerepost', 'enten...",0.222636,-0.364943,-0.140176,-0.099798,0.170952,-0.180010,0.353187,0.602269,0.081814,...,-0.025534,0.697447,0.132731,0.187729,0.285927,0.120604,-0.354065,0.044958,-0.431190,0
3,"['minuto', 'touro', 'ouro']",0.019667,0.067234,-0.117171,-0.004117,-0.016320,-0.133322,0.007701,0.329346,-0.013447,...,-0.032330,0.265353,0.214843,0.206906,0.149337,-0.142627,-0.299524,0.294996,-0.002506,1
4,"['Ricktolledo', 'sim']",0.304922,-0.007242,-0.248962,-0.251848,0.077637,-0.208594,-0.040489,0.165175,0.095193,...,0.029760,0.275768,0.157836,0.166802,0.105236,-0.017478,-0.224127,0.182770,-0.183279,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12169,"['um', 'noite', 'encontro', 'muito', 'conhecim...",0.385639,-0.163126,-0.161703,-0.347907,0.063867,-0.230677,0.215600,0.676717,0.157806,...,-0.184806,1.248494,0.181885,0.419142,0.284126,0.237861,-0.622363,0.271487,-0.428794,0
12170,"['erro', 'financeiro', 'eliminar', 'antes', 'd...",0.501189,-0.143322,-0.599311,-0.384006,0.207775,-0.374636,0.072794,0.934155,0.189281,...,0.132775,0.808506,0.583185,0.393526,0.363416,-0.023957,-0.783997,0.626378,-0.398472,0
12171,"['estar', 'muito', 'grato', 'todo', 'esforco',...",0.574576,0.287958,-0.388242,-0.518758,0.179550,-0.304643,-0.059453,0.401551,0.203472,...,-0.121086,0.782358,0.479056,0.613490,0.233333,-0.014855,-0.657147,0.473138,-0.245109,1
12172,['dorsodamaocomdedoindicadorapontandoparaadire...,0.304480,0.013746,-0.179800,-0.215241,0.065398,-0.160660,0.023386,0.475333,0.077669,...,-0.045335,0.722488,0.201854,0.270953,0.163493,0.029630,-0.392153,0.240040,-0.196458,0


In [7]:
df_vec = df_vec.dropna()

In [157]:
print(df_vec.isnull().sum())  # Verificar valores nulos
print(df_vec.isna().sum())  # Verificar valores vazios

tokens_lema    0
vec1           0
vec2           0
vec3           0
vec4           0
              ..
vec147         0
vec148         0
vec149         0
vec150         0
sentimento     0
Length: 152, dtype: int64
tokens_lema    0
vec1           0
vec2           0
vec3           0
vec4           0
              ..
vec147         0
vec148         0
vec149         0
vec150         0
sentimento     0
Length: 152, dtype: int64


In [23]:
from sklearn.model_selection import train_test_split
dropar = df_vec[['tokens_lema', 'sentimento']]
X = df_vec.drop(columns = dropar)
y = df_vec['sentimento']
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Naive Bayes
classifier = GaussianNB()

# Treino de classificação
classifier.fit(X_train, y_train)

# Predição do teste
y_pred = classifier.predict(X_test)

# Validando modelo
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)


Accuracy: 0.5552361396303901
Classification Report:
              precision    recall  f1-score   support

          -1       0.39      0.80      0.53       513
           0       0.80      0.48      0.60      1073
           1       0.57      0.50      0.53       849

    accuracy                           0.56      2435
   macro avg       0.59      0.59      0.55      2435
weighted avg       0.63      0.56      0.56      2435



In [24]:
# Calcule a acurácia nos dados de treinamento
train_accuracy = classifier.score(X_train, y_train)

# Calcule a acurácia nos dados de teste
test_accuracy = classifier.score(X_test, y_test)

# Imprima as acurácias
print("Acurácia de treinamento:", train_accuracy)
print("Acurácia de teste:", test_accuracy)

Acurácia de treinamento: 0.5484135948249307
Acurácia de teste: 0.5552361396303901


In [12]:
!pip install catboost

In [25]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

# Crie uma instância do modelo CatBoost
classifier = CatBoostClassifier()

# Defina um dicionário de hiperparâmetros a serem ajustados
param_grid = {
    'learning_rate': [0.01, 0.1, 1.0]
}

# Crie uma instância de GridSearchCV
grid_search = GridSearchCV(classifier, param_grid=param_grid, cv=5)

# Treine o modelo com a busca em grade
grid_search.fit(X_train, y_train)

# Obtenha a melhor combinação de hiperparâmetros encontrada pela busca em grade
best_params = grid_search.best_params_

# Use o modelo com os melhores hiperparâmetros para fazer previsões no conjunto de testes
y_pred = grid_search.predict(X_test)

0:	learn: 1.0932849	total: 51ms	remaining: 50.9s
1:	learn: 1.0880101	total: 96.5ms	remaining: 48.1s
2:	learn: 1.0830022	total: 142ms	remaining: 47.2s
3:	learn: 1.0776729	total: 189ms	remaining: 47.2s
4:	learn: 1.0727595	total: 236ms	remaining: 47s
5:	learn: 1.0678277	total: 289ms	remaining: 47.9s
6:	learn: 1.0631277	total: 337ms	remaining: 47.8s
7:	learn: 1.0584362	total: 384ms	remaining: 47.6s
8:	learn: 1.0536439	total: 436ms	remaining: 48s
9:	learn: 1.0490025	total: 488ms	remaining: 48.3s
10:	learn: 1.0445522	total: 536ms	remaining: 48.2s
11:	learn: 1.0402507	total: 582ms	remaining: 48s
12:	learn: 1.0357453	total: 629ms	remaining: 47.8s
13:	learn: 1.0315123	total: 677ms	remaining: 47.7s
14:	learn: 1.0273889	total: 738ms	remaining: 48.5s
15:	learn: 1.0232898	total: 785ms	remaining: 48.3s
16:	learn: 1.0192434	total: 834ms	remaining: 48.2s
17:	learn: 1.0152325	total: 885ms	remaining: 48.3s
18:	learn: 1.0114251	total: 937ms	remaining: 48.4s
19:	learn: 1.0074153	total: 998ms	remaining: 48

164:	learn: 0.7742843	total: 8.28s	remaining: 41.9s
165:	learn: 0.7737073	total: 8.33s	remaining: 41.8s
166:	learn: 0.7730943	total: 8.38s	remaining: 41.8s
167:	learn: 0.7723953	total: 8.42s	remaining: 41.7s
168:	learn: 0.7717647	total: 8.47s	remaining: 41.7s
169:	learn: 0.7711515	total: 8.53s	remaining: 41.6s
170:	learn: 0.7705023	total: 8.57s	remaining: 41.6s
171:	learn: 0.7698970	total: 8.62s	remaining: 41.5s
172:	learn: 0.7693018	total: 8.68s	remaining: 41.5s
173:	learn: 0.7687526	total: 8.73s	remaining: 41.4s
174:	learn: 0.7681564	total: 8.79s	remaining: 41.4s
175:	learn: 0.7675703	total: 8.84s	remaining: 41.4s
176:	learn: 0.7670802	total: 8.89s	remaining: 41.3s
177:	learn: 0.7665016	total: 8.94s	remaining: 41.3s
178:	learn: 0.7658824	total: 8.99s	remaining: 41.2s
179:	learn: 0.7653747	total: 9.04s	remaining: 41.2s
180:	learn: 0.7648123	total: 9.09s	remaining: 41.1s
181:	learn: 0.7641990	total: 9.13s	remaining: 41s
182:	learn: 0.7635752	total: 9.18s	remaining: 41s
183:	learn: 0.76

323:	learn: 0.7147659	total: 16.1s	remaining: 33.7s
324:	learn: 0.7145254	total: 16.2s	remaining: 33.6s
325:	learn: 0.7142096	total: 16.2s	remaining: 33.6s
326:	learn: 0.7139710	total: 16.3s	remaining: 33.5s
327:	learn: 0.7137196	total: 16.3s	remaining: 33.5s
328:	learn: 0.7135492	total: 16.4s	remaining: 33.4s
329:	learn: 0.7133009	total: 16.4s	remaining: 33.4s
330:	learn: 0.7130448	total: 16.5s	remaining: 33.3s
331:	learn: 0.7128243	total: 16.5s	remaining: 33.3s
332:	learn: 0.7125887	total: 16.6s	remaining: 33.2s
333:	learn: 0.7123489	total: 16.6s	remaining: 33.2s
334:	learn: 0.7121365	total: 16.7s	remaining: 33.1s
335:	learn: 0.7119501	total: 16.7s	remaining: 33.1s
336:	learn: 0.7116412	total: 16.8s	remaining: 33s
337:	learn: 0.7114425	total: 16.8s	remaining: 33s
338:	learn: 0.7111615	total: 16.9s	remaining: 32.9s
339:	learn: 0.7109203	total: 16.9s	remaining: 32.9s
340:	learn: 0.7106365	total: 17s	remaining: 32.8s
341:	learn: 0.7104331	total: 17s	remaining: 32.8s
342:	learn: 0.710239

485:	learn: 0.6849332	total: 23.9s	remaining: 25.3s
486:	learn: 0.6847869	total: 23.9s	remaining: 25.2s
487:	learn: 0.6846639	total: 24s	remaining: 25.2s
488:	learn: 0.6845303	total: 24s	remaining: 25.1s
489:	learn: 0.6844023	total: 24.1s	remaining: 25.1s
490:	learn: 0.6842197	total: 24.1s	remaining: 25s
491:	learn: 0.6840721	total: 24.2s	remaining: 25s
492:	learn: 0.6839427	total: 24.2s	remaining: 24.9s
493:	learn: 0.6837907	total: 24.3s	remaining: 24.9s
494:	learn: 0.6836075	total: 24.3s	remaining: 24.8s
495:	learn: 0.6834551	total: 24.4s	remaining: 24.8s
496:	learn: 0.6832487	total: 24.4s	remaining: 24.7s
497:	learn: 0.6831259	total: 24.5s	remaining: 24.7s
498:	learn: 0.6829603	total: 24.5s	remaining: 24.6s
499:	learn: 0.6828291	total: 24.5s	remaining: 24.5s
500:	learn: 0.6826343	total: 24.6s	remaining: 24.5s
501:	learn: 0.6825348	total: 24.6s	remaining: 24.4s
502:	learn: 0.6823435	total: 24.7s	remaining: 24.4s
503:	learn: 0.6822451	total: 24.7s	remaining: 24.3s
504:	learn: 0.682082

644:	learn: 0.6638511	total: 31.6s	remaining: 17.4s
645:	learn: 0.6636946	total: 31.6s	remaining: 17.3s
646:	learn: 0.6636073	total: 31.7s	remaining: 17.3s
647:	learn: 0.6634645	total: 31.7s	remaining: 17.2s
648:	learn: 0.6633220	total: 31.8s	remaining: 17.2s
649:	learn: 0.6631955	total: 31.8s	remaining: 17.1s
650:	learn: 0.6630937	total: 31.9s	remaining: 17.1s
651:	learn: 0.6629566	total: 31.9s	remaining: 17s
652:	learn: 0.6628062	total: 32s	remaining: 17s
653:	learn: 0.6627246	total: 32s	remaining: 16.9s
654:	learn: 0.6626210	total: 32.1s	remaining: 16.9s
655:	learn: 0.6625134	total: 32.1s	remaining: 16.8s
656:	learn: 0.6624015	total: 32.2s	remaining: 16.8s
657:	learn: 0.6622626	total: 32.2s	remaining: 16.8s
658:	learn: 0.6621540	total: 32.3s	remaining: 16.7s
659:	learn: 0.6619888	total: 32.3s	remaining: 16.7s
660:	learn: 0.6618133	total: 32.4s	remaining: 16.6s
661:	learn: 0.6616935	total: 32.4s	remaining: 16.6s
662:	learn: 0.6615729	total: 32.5s	remaining: 16.5s
663:	learn: 0.661532

...

241:	learn: 0.5284286	total: 18.1s	remaining: 56.6s
242:	learn: 0.5278109	total: 18.2s	remaining: 56.6s
243:	learn: 0.5269219	total: 18.2s	remaining: 56.5s
244:	learn: 0.5263583	total: 18.3s	remaining: 56.5s
245:	learn: 0.5258293	total: 18.4s	remaining: 56.4s
246:	learn: 0.5250173	total: 18.5s	remaining: 56.3s
247:	learn: 0.5242746	total: 18.5s	remaining: 56.2s
248:	learn: 0.5233635	total: 18.6s	remaining: 56.2s
249:	learn: 0.5228004	total: 18.7s	remaining: 56.1s
250:	learn: 0.5223592	total: 18.8s	remaining: 56s
251:	learn: 0.5219981	total: 18.8s	remaining: 55.9s
252:	learn: 0.5215533	total: 18.9s	remaining: 55.8s
253:	learn: 0.5207816	total: 19s	remaining: 55.7s
254:	learn: 0.5203688	total: 19.1s	remaining: 55.7s
255:	learn: 0.5198524	total: 19.1s	remaining: 55.6s
256:	learn: 0.5191746	total: 19.2s	remaining: 55.5s
257:	learn: 0.5184591	total: 19.3s	remaining: 55.5s
258:	learn: 0.5176718	total: 19.4s	remaining: 55.5s
259:	learn: 0.5170176	total: 19.5s	remaining: 55.4s
260:	learn: 0.51

402:	learn: 0.4429356	total: 29.8s	remaining: 44.1s
403:	learn: 0.4422284	total: 29.9s	remaining: 44s
404:	learn: 0.4417285	total: 29.9s	remaining: 44s
405:	learn: 0.4411933	total: 30s	remaining: 43.9s
406:	learn: 0.4406641	total: 30.1s	remaining: 43.8s
407:	learn: 0.4399881	total: 30.2s	remaining: 43.8s
408:	learn: 0.4394880	total: 30.2s	remaining: 43.7s
409:	learn: 0.4392847	total: 30.3s	remaining: 43.6s
410:	learn: 0.4389674	total: 30.4s	remaining: 43.5s
411:	learn: 0.4385907	total: 30.4s	remaining: 43.4s
412:	learn: 0.4383822	total: 30.5s	remaining: 43.4s
413:	learn: 0.4380774	total: 30.6s	remaining: 43.3s
414:	learn: 0.4376537	total: 30.7s	remaining: 43.2s
415:	learn: 0.4372923	total: 30.8s	remaining: 43.2s
416:	learn: 0.4367370	total: 30.8s	remaining: 43.1s
417:	learn: 0.4362268	total: 30.9s	remaining: 43s
418:	learn: 0.4356728	total: 31s	remaining: 43s
419:	learn: 0.4350206	total: 31.1s	remaining: 42.9s
420:	learn: 0.4345697	total: 31.1s	remaining: 42.8s
421:	learn: 0.4342580	to

561:	learn: 0.3774044	total: 41.4s	remaining: 32.3s
562:	learn: 0.3770326	total: 41.5s	remaining: 32.2s
563:	learn: 0.3767128	total: 41.5s	remaining: 32.1s
564:	learn: 0.3764070	total: 41.6s	remaining: 32s
565:	learn: 0.3758354	total: 41.7s	remaining: 32s
566:	learn: 0.3752948	total: 41.7s	remaining: 31.9s
567:	learn: 0.3748373	total: 41.8s	remaining: 31.8s
568:	learn: 0.3744804	total: 41.9s	remaining: 31.7s
569:	learn: 0.3740945	total: 41.9s	remaining: 31.6s
570:	learn: 0.3738846	total: 42s	remaining: 31.6s
571:	learn: 0.3734038	total: 42.1s	remaining: 31.5s
572:	learn: 0.3729453	total: 42.2s	remaining: 31.4s
573:	learn: 0.3725322	total: 42.2s	remaining: 31.3s
574:	learn: 0.3720289	total: 42.3s	remaining: 31.3s
575:	learn: 0.3715087	total: 42.4s	remaining: 31.2s
576:	learn: 0.3711731	total: 42.4s	remaining: 31.1s
577:	learn: 0.3709650	total: 42.5s	remaining: 31s
578:	learn: 0.3705733	total: 42.6s	remaining: 31s
579:	learn: 0.3701936	total: 42.6s	remaining: 30.9s
580:	learn: 0.3700051	

720:	learn: 0.3255909	total: 53.1s	remaining: 20.5s
721:	learn: 0.3253419	total: 53.2s	remaining: 20.5s
722:	learn: 0.3250063	total: 53.2s	remaining: 20.4s
723:	learn: 0.3247152	total: 53.3s	remaining: 20.3s
724:	learn: 0.3243889	total: 53.4s	remaining: 20.2s
725:	learn: 0.3240263	total: 53.4s	remaining: 20.2s
726:	learn: 0.3238899	total: 53.5s	remaining: 20.1s
727:	learn: 0.3235761	total: 53.6s	remaining: 20s
728:	learn: 0.3234247	total: 53.6s	remaining: 19.9s
729:	learn: 0.3231573	total: 53.7s	remaining: 19.9s
730:	learn: 0.3229659	total: 53.8s	remaining: 19.8s
731:	learn: 0.3224862	total: 53.8s	remaining: 19.7s
732:	learn: 0.3223086	total: 53.9s	remaining: 19.6s
733:	learn: 0.3219343	total: 54s	remaining: 19.6s
734:	learn: 0.3215823	total: 54s	remaining: 19.5s
735:	learn: 0.3212747	total: 54.1s	remaining: 19.4s
736:	learn: 0.3210242	total: 54.2s	remaining: 19.3s
737:	learn: 0.3208142	total: 54.2s	remaining: 19.3s
738:	learn: 0.3206173	total: 54.3s	remaining: 19.2s
739:	learn: 0.3204

882:	learn: 0.2843761	total: 1m 4s	remaining: 8.56s
883:	learn: 0.2843152	total: 1m 4s	remaining: 8.49s
884:	learn: 0.2841139	total: 1m 4s	remaining: 8.42s
885:	learn: 0.2840256	total: 1m 4s	remaining: 8.34s
886:	learn: 0.2837235	total: 1m 4s	remaining: 8.27s
887:	learn: 0.2836281	total: 1m 5s	remaining: 8.2s
888:	learn: 0.2833660	total: 1m 5s	remaining: 8.13s
889:	learn: 0.2830928	total: 1m 5s	remaining: 8.05s
890:	learn: 0.2829867	total: 1m 5s	remaining: 7.98s
891:	learn: 0.2828603	total: 1m 5s	remaining: 7.91s
892:	learn: 0.2827311	total: 1m 5s	remaining: 7.83s
893:	learn: 0.2825430	total: 1m 5s	remaining: 7.76s
894:	learn: 0.2823942	total: 1m 5s	remaining: 7.69s
895:	learn: 0.2821854	total: 1m 5s	remaining: 7.61s
896:	learn: 0.2820558	total: 1m 5s	remaining: 7.54s
897:	learn: 0.2818951	total: 1m 5s	remaining: 7.47s
898:	learn: 0.2817838	total: 1m 5s	remaining: 7.39s
899:	learn: 0.2815394	total: 1m 5s	remaining: 7.32s
900:	learn: 0.2813772	total: 1m 5s	remaining: 7.25s
901:	learn: 0

In [14]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Accuracy: 0.7215605749486653
Classification Report:
              precision    recall  f1-score   support

          -1       0.64      0.64      0.64       501
           0       0.76      0.75      0.76      1038
           1       0.73      0.73      0.73       896

    accuracy                           0.72      2435
   macro avg       0.71      0.71      0.71      2435
weighted avg       0.72      0.72      0.72      2435



In [27]:
# Calcule a acurácia nos dados de treinamento
train_accuracy = grid_search.score(X_train, y_train)

# Calcule a acurácia nos dados de teste
test_accuracy = grid_search.score(X_test, y_test)

# Imprima as acurácias
print("Acurácia de treinamento:", train_accuracy)
print("Acurácia de teste:", test_accuracy)
#Pelo Resultado, teremos que balancear os dados

Acurácia de treinamento: 0.9519457849881918
Acurácia de teste: 0.7232032854209446


In [28]:
from sklearn.metrics import confusion_matrix
# Crie a matriz de confusão
confusion_mtx = confusion_matrix(y_test, y_pred)

# Imprima a matriz de confusão
print(confusion_mtx)

[[317  77 119]
 [ 93 801 179]
 [ 82 124 643]]
